# Österreich: Vermischtes Monitoring zu Gesundheits- & SARS-CoV-2-Daten

Erstellt aus <https://github.com/zeitferne/covidat-tools/blob/main/notebooks/monitoring.ipynb>

Inhalt:

* [Medikamentenmangel](#Medikamentenmangel)
* [SARS-CoV-2-Abwassermonitoring](#SARS-CoV-2-Abwassermonitoring)
* [Krankenstand](#Krankenstand)
* [Influenza und Atemwegserkrankungen](#Influenza-und-Atemwegserkrankungen)

In [ ]:
#@export: --no-input

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, "de_AT.UTF-8");

In [ ]:
%matplotlib inline
%precision %.4f
#%load_ext snakeviz
import colorcet
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates
import matplotlib.ticker
import matplotlib.colors
import seaborn as sns
from importlib import reload
from datetime import date, datetime, timedelta
from IPython.display import display, Markdown, HTML
import re

from covidat import util, covdata, cov, collectshortage

pd.options.display.precision = 4

plt.rcParams['figure.figsize'] = (16*0.7,9*0.7)
plt.rcParams['figure.dpi'] =  120 #80
plt.rcParams['figure.facecolor'] = '#fff'
sns.set_theme(style="whitegrid")
sns.set_palette('tab10')

plt.rcParams['image.cmap'] = cov.un_cmap

pd.options.display.max_rows = 120
pd.options.display.min_rows = 40

In [ ]:
display(Markdown(f"Zuletzt erstellt am *{datetime.now().strftime('%a %d.%m.%Y, nach %H Uhr')}* (Daten können älter sein, siehe einzelne Diagramme)"))

# Medikamentenmangel

* Daten von: Österreichiches Bundesamt für Sicherheit im Gesundheitswesen (BASG)
* Daten-URLs:
  * <https://medicineshortage.basg.gv.at/vertriebseinschraenkungen> (Webansicht und xlsx)
  * <https://www.basg.gv.at/fuer-unternehmen/datenbereitstellung-vertriebseinschraenkungen> (xml)
* Daten-Aktualisierung: Täglich (mit sporadischen Aussetzern, v.a. an Nicht-Werktagen)
* [Mehr Daten-Infos](https://github.com/zeitferne/covidat-data/blob/main/docs/basg-medicineshortage.md)

In [ ]:
medshort = pd.read_csv(
    util.COLLECTROOT / "medshort/medshort.csv",
    sep=";")
medshort["Datum"] = cov.parseutc_at(medshort["Datum"], format=cov.ISO_DATE_FMT)
medshort.set_index("Datum", inplace=True)
if False:
    fig, ax = plt.subplots()
    ax.plot(medshort["NLimitedMeds"], marker="o")
    ax.set_ylim(bottom=0, top=medshort["NLimitedMeds"].max() * 1.05)
    cov.set_date_opts(ax)
    fig.suptitle("Anzahl Einträge in Liste der Meldungen zu Vertriebseinschränkungen von Arzneispezialitäten")
    ax.set_title("Quelle: medicineshortage.basg.gv.at/vertriebseinschraenkungen")

In [ ]:
def labelavail(avail):
    return (avail
        .replace("verfügbar gemäß §4 (1)", "§4(1) (~Mangel entgegen Herstellerangabe)")
        .replace("teilweise verfügbar", "gewisse Größen voll verfügbar (Verwechslung mit eing. verf. möglich)"))

In [ ]:
med_x = pd.read_csv(util.COLLECTROOT / "medshort/medshort_ex.csv", encoding="utf-8", sep=";"
                   ).query("Datum >= '2022-12-01' and (SourceFormat == 'xml' or Datum <= '2023-06-25')")
display(med_x.query("Availability == 'verfügbar'"))
med_x = med_x.query("Availability != 'verfügbar'")
med_x["Datum"] = pd.to_datetime(med_x["Datum"], format="%Y-%m-%d")
med_x["Datum"].freq = "D"
#med_x["Datum_n"] = matplotlib.dates.date2num(med_x["Datum"])
AvailC = pd.CategoricalDtype([
    "nicht verfügbar",
    "eingeschränkt verfügbar",
    "verfügbar gemäß §4 (1)",
    "teilweise verfügbar"])
med_x["Avail_c"] = med_x["Availability"].astype(AvailC)
med_x.sort_values(["Datum", "Avail_c"], kind="stable", inplace=True)
#display(med_x)
med_xh = med_x.query("Usage == 'Human'").reset_index()
#ax = sns.area(med_xh, x="Datum", y="N", hue="Availability", marker="o")
#ax = med_xh.pivot(index="Datum", columns="Availability", values="N").resample("D").plot.bar(stacked=True)
#cov.set_date_opts(ax)
#ax.set_ylim(bottom=0)
fig, ax = plt.subplots()
ax.set_axisbelow(False)
#ax.plot(medshort["NLimitedMeds"], marker="o", color="grey",
#        label="Anzahl Einträge insgesamt, inkl. Veterinär: " + str(medshort["NLimitedMeds"].iloc[-1]), zorder=1)
pltr = cov.StackBarPlotter(
    ax, pd.date_range(start=med_xh["Datum"].min(), end=med_xh["Datum"].max(), freq="1D"),
    lw=0, width=1)
ax.set_prop_cycle(color=sns.color_palette("plasma", n_colors=med_x["Avail_c"].nunique()))
for avail, avail_d in med_xh.groupby("Availability", sort=False):
    #display(avail_d)
    pltr(avail_d.set_index("Datum")["N"].resample("1D").interpolate(limit=1).to_numpy(),
         label=labelavail(avail) + ": " + str(avail_d["N"].iloc[-1]))
    
def decorate_medshort(ax, titley, supy, extralabel=""):
    ax.figure.suptitle("Medikamentenmangel in Österreich", y=supy)
    cov.set_date_opts(ax, showyear=True)
    ax.axvline(pd.to_datetime("2023-06-23T12:00:00"), color="k", ls=":",
           label="Umstellung eigener Datensammlung von xlsx auf xml")
    ax.set_title(
    "Darstellung: @zeitferne | Daten: medicineshortage.basg.gv.at/vertriebseinschraenkungen, bis "
    + med_xh.iloc[-1]["Datum"].strftime("%d.%m.%Y")
   # + "\nVerschiedene Packungsgrößen (nicht Dosisstärken) möglichst als gleiches Medikament gezählt"
    + extralabel,
    y=titley)
    ax.set_ylabel("Anzahl Human-Medikamente")
#ax.figure.autofmt_xdate()
decorate_medshort(ax, 1.05, 1.01, " | 1-tägige Lücken interpoliert")
ax.legend(loc="lower left", title="Anzahl Human-Medikamente je Kategorie")
#med_xh

if False:
    ax.text(
        0.5, 1.01,
        "Falls ein Medikament in allen beschränkten Dosisstärken in allen Packungsgrößen beschränkt ist, "
        "aber eine andere Dosisstärke uneingeschränkt verfügbar wäre,\n"
        "wird stattdessen wegen Limitierungen in der Daten-Analyse fälschlicherweise "
        "die stärkere Beschränkung angegeben.",
        fontsize="x-small",
        ha="center",
        transform=ax.transAxes)
display(cov.filterlatest(med_xh))
display(cov.filterlatest(med_xh)["N"].sum())
#display(med_xh[med_xh["Datum"] == med_xh["Datum"].min()])

In [ ]:
azr = collectshortage.load_azr()
sx = collectshortage.load_veasp_xml(
    util.DATAROOT / "basg-medicineshortage/VertriebseinschraenkungenASP_latest.xml", azr)

In [ ]:
#fig, axs = plt.subplots(ncols=2, sharey=True, gridspec_kw={'width_ratios':[2,1])
fig, ax = plt.subplots(figsize=(5, 5))
sx0 = sx.query("Status != 'verfügbar'").copy()
sx0["Grund"].fillna("?", inplace=True)
ax = sx0.value_counts("Grund", ascending=True, dropna=False).plot.barh()
ax.bar_label(ax.containers[0])
fig = ax.figure
fig.suptitle("Ans BASG gemeldeter Grund für Mangel (alle Verfügbarkeits-Kategorien)", y=0.93, x=0);
#ax = sx.query("Status == 'nicht verfügbar'").value_counts("Grund", ascending=True).plot.barh(ax=axs[1])
#ax.bar_label(ax.containers[0])

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax = sx0.query("Status == 'nicht verfügbar'").value_counts("Grund", ascending=True, dropna=False).plot.barh()
ax.bar_label(ax.containers[0])
fig = ax.figure
fig.suptitle("Ans BASG gemeldeter Grund für Mangel (nur \"nicht verfügbar\")", y=0.93, x=0);

In [ ]:
fig, ax = plt.subplots(figsize=(5, 8))
query = "Grund == 'Erhöhter Mehrbedarf'"
fig.suptitle(query + "\nje Wirkstoff (Mehrfachzählung bei mehreren Wirkstoffen, nur >1)",
            x=0)
(sx
 .query(query)
 .explode("Wirkstoffe")
 .value_counts("Wirkstoffe", ascending=True, dropna=False)
 .where(lambda x: x > 1)
 .dropna()
 .plot.barh());

In [ ]:
fig, ax = plt.subplots(figsize=(5, 8))
query = "Grund == 'Erhöhter Mehrbedarf' and Status == 'nicht verfügbar'"
fig.suptitle(query + "\nje Wirkstoff (Mehrfachzählung bei mehreren Wirkstoffen)",
            x=0)
(sx
 .query(query)
 .explode("Wirkstoffe")
 .value_counts("Wirkstoffe", ascending=True, dropna=False)
 .plot.barh())

In [ ]:
sx0 = sx.copy()
sx0["Wirkstoffe"] = sx0["Wirkstoffe"].str.join(", ")
sx0.to_csv(util.COLLECTROOT / "medshort/VertriebseinschraenkungenASP_agg.csv", sep=";", encoding="utf-8-sig", index=False)

In [ ]:
pdata = med_xh.copy()
med_xh["Availability"] = labelavail(med_xh["Availability"])
ax = sns.lineplot(med_xh, x="Datum", y="N", hue="Availability")
ax.get_legend().remove()
decorate_medshort(ax, supy=1.065, titley=1.14)
fig = ax.figure
fig.legend(*cov.sortedlabels(ax, med_xh, "N", "Availability", fmtval=lambda v: f"{v:.0f}"),
           frameon=False, ncol=2, loc="upper center", bbox_to_anchor=(0.5, 1))
ax.set_ylim(bottom=0);

# SARS-CoV-2-Abwassermonitoring

* Daten von: hydro-IT GmbH, Bundesministerium für Soziales, Gesundheit, Pflege und Konsumentenschutz (BMSGPK)
* Daten-URL: <https://abwassermonitoring.at/dashboard/>
* Daten-Aktualisierung: Mehrmals wöchentlich, üblicherweise täglich von Dienstag bis inkl. Samstag
  (Dienstags meist nur minimale Korrekturen)

In [ ]:
blverlauf = covdata.load_ww_blverlauf()
blvfull = (
        blverlauf[blverlauf["Datum"] >= pd.to_datetime("2022-09-01")])
abwfrom = blverlauf["FileDate"].min()
blverlauf_last = blverlauf[blverlauf["FileDate"] == blverlauf["FileDate"].max()]
blv1 = covdata.first_filedate(blvfull, ["Bundesland"])
ABW_TINFO = (
    f"Darstellung: @zeitferne | Daten: abwassermonitoring.at/dashboard letzte Änderung"
    f" {blverlauf_last.iloc[-1]['FileDate'].tz_localize('UTC').tz_convert('Europe/Vienna').strftime('%x %H:%M')}")

In [ ]:
if False:
    pdata = blverlauf_last#.query("Datum >= '2022-05-15'")
    ax = sns.lineplot(data=pdata, x="Datum", y="coverage", hue="Bundesland", hue_order=cov.SHORTNAME_BY_BUNDESLAND.keys())
    cov.set_percent_opts(ax)
    ax.legend(*cov.sortedlabels(ax, pdata, "coverage", fmtval=lambda v: f"{v:.0%}"),
              ncol=2, fontsize="x-small", loc="upper left")
    ax.figure.suptitle("COVID-Abwassermonitoring-Abdeckung")
    ax.set_title(ABW_TINFO)
    ax.set_ylim(bottom=0)
    cov.set_date_opts(ax)
    cov.labelend2(ax, pdata, "coverage")

In [ ]:
for selbl in ("Österreich", "Wien"): #blverlauf["Bundesland"].unique():
#selbl="Österreich"
    pdata = (blverlauf
        [blverlauf["Datum"] >= pd.to_datetime("2023-04-01")]
             .query(f"Bundesland == '{selbl}'")).copy()
    bv1i = blv1.xs(selbl, level="Bundesland").reset_index()[["FileDate", "Datum"]]
    pdi = pdata.reset_index().merge(bv1i, how="inner").set_index('index')
    pdata["sz"] = False
    pdata.loc[pdi.index, "sz"] = True
    plt.figure()
    ax = sns.lineplot(data=pdata,
                      palette="plasma_r",
                      x="Datum", y="y", mew=0, hue="FileDate",
                      legend=False,
                      lw=0.7,
                      alpha=0.7
                     )
    ax = sns.lineplot(data=pdata.where(pdata["sz"]),
                      palette="plasma_r",
                      x="Datum", y="y", mew=0, hue="FileDate",
                      legend=False, lw=2, marker=".",
                     )
    cov.set_date_opts(ax, pdata["Datum"])
    ax.set_xlim(left=pdata["Datum"].min())
    ax.set_ylabel("Genkopien pro Tag / EW * 10⁶")
    ax.figure.suptitle(selbl + ": SARS-CoV-2 im Abwasser, verschiedene Datenstände (logarithmisch)")
    ax.set_title(ABW_TINFO + ", Daten bis: " + pdata.iloc[-1]["Datum"].strftime("%x"))
    cov.set_logscale(ax)
    #ax.yaxis.set_major_locator(matplotlib.ticker.AutoLocator())
    #ax.set_ylim(bottom=0)


In [ ]:
from itertools import pairwise
blvat = blverlauf.query("Bundesland == 'Österreich'").copy()
assert len(blvat) > 0
grp = blvat.groupby("FileDate")
samevals = [False] + [len(fd1) == len(fd2) and (fd1.to_numpy() == fd2.to_numpy()).all()
 for (_, fd1), (_, fd2) in pairwise(grp["y"])]
samevals = pd.Series(samevals, index=grp.indices, dtype=bool, name="same")
blvat = blvat.merge(samevals, left_on="FileDate", right_index=True)
ax = sns.lineplot(data=blvat[~blvat["same"]],
                      palette="cet_glasbey_dark",
                      x="Datum", y="y", mew=0, hue="FileDate",
                      legend=False, hue_norm=matplotlib.colors.Normalize())
fig = ax.figure
fig.suptitle("Abwasser: Alle Datenstände Österreich")

In [ ]:
def draw_bl_ww_range(selbl, ax, logscale=True, fromdate=None, showcases=False):
    blvfull_bl = blvfull[blvfull["Bundesland"] == selbl]
    if fromdate:
        blvfull_bl = blvfull_bl[blvfull_bl["Datum"] >= fromdate]
    blvaext = (
        blvfull_bl
        .groupby(["Datum", "Bundesland"])
        .agg({"y": ["min", "max", "last", "first"], "FileDate": ["first", "last"]})
        .xs(selbl, level="Bundesland"))
    ##fig, ax = plt.subplots()
    if logscale:
        cov.set_logscale(ax)
    ax.fill_between(blvaext.index, blvaext[("y", "min")], blvaext[("y", "max")], alpha=0.3,
                    label="Minimum‒Maximum")
    ylast = blvaext[blvaext[("FileDate", "last")] == blverlauf["FileDate"].max()][("y", "last")]
    ax.plot(
        ylast,
        color="k", label="Letzter Stand")
    ax.plot(blvfull_bl[blvfull_bl["FileDate"] == blvfull_bl["FileDate"].unique()[-2]].set_index("Datum")["y"],
            ls="--", zorder=5, alpha=0.8, lw=0.7, color="k", label="Vorheriger Stand (falls abweichend)")
    
    #display(blvaext)
    first = True
    for _, values in blvaext.groupby(("FileDate", "first")):
        ax.plot(values[("y", "first")],
                marker="." if len(values) <= 1 else None,
                #markersize=,
                mew=0,
                color="C0",
                label="Erstmeldung" if first else None,
                lw=1.8
               )
        first = False
    ax.plot(blvaext[("y", "first")], color="C0", lw=0.8)
    ax.set_facecolor("none")
    ##cov.set_date_opts(ax, blvaext.index.get_level_values("Datum"))
    ##ax.set_xlim(left=blvaext.index.get_level_values("Datum").min())
    ##ax.set_ylabel("Genkopien pro Tag / EW * 10⁶")
    ##ax.set_title(ABW_TINFO + ", Daten bis " + blvaext.index[-1].strftime("%a %x"), y=1.07)
    ##ax.set_xlim(*ax.get_xlim())
    if showcases:
        ax2 = ax.twinx()
        ax2.set_zorder(-3)
        if logscale:
            cov.set_logscale(ax2)
        inz = fs[(fs["Bundesland"] == selbl) & (
            fs["Datum"] >= blvaext.index[0])].set_index("Datum")["inz"]
        inzmax = inz.iloc[:90].max()
        ymax = ylast.iloc[:90].max()
        inzpery = inzmax / ymax
        ax2.plot(inz, color="C1",
            label="7-Tage-Inzidenz (rechts)" + AGES_STAMP)
        ax2.grid(False)
        def llim(x, xb): return x * 0.9 if logscale else x - xb
        def ulim(x, xb): return x * 1.1 if logscale else x + xb
        inzb = abs(ylast.min() - ylast.max()) * inzpery * 0.15
        
        if not logscale:
            ax2.set_ylim(bottom=0)
            ax.set_ylim(bottom=0)
        
        inzlims = ax2.get_ylim()
        ylims = ax.get_ylim()
        #print(selbl, inzpery, inzlims, ylims)
        inzlim_r = (min(ylims[0] * inzpery, inzlims[0]), max(ylims[1] * inzpery, inzlims[1]))
        ylim_r = (min(inzlim_r[0] / inzpery, ylims[0]), max(inzlim_r[1] / inzpery, ylims[1]))
        inzlim_r = (min(ylim_r[0] * inzpery, inzlim_r[0]), max(ylim_r[1] * inzpery, inzlim_r[1]))
        ax.set_ylim(*ylim_r)
        ax2.set_ylim(*inzlim_r)
        
        if False:
            ax2.set_ylim(
                min(llim(blvaext[("y", "min")].min() * inzpery, inzb), ax2.get_ylim()[0]),
                max(ulim(blvaext[("y", "max")].max() * inzpery, inzb), ax2.get_ylim()[1]),
            )
            yb = abs(inz.min() - inz.max()) / inzpery * 0.15
            ax.set_ylim(
                min(llim(inz.min() / inzpery, yb), ax.get_ylim()[0]),
                max(ulim(inz.max() / inzpery, yb), ax.get_ylim()[1]),
            )
    else:
        ax2 = None
      
        
        #ax2.set_ylim(bottom=0)
        #ax2.set_ylabel("Hospitalisierte CoV-PatientInnen")
    ##fig.legend(ncol=4, frameon=False, loc="upper center", bbox_to_anchor=(0.5, 0.95))
    #break
    #ax.set_xlim(
    #    left=fromdate,
    #    right=maxdate + max(2, timedelta(ndays * 0.05)))
    return ax2, blvaext

def draw_each_bl_ww(logscale=True, ndays=None):
    maxdate = blverlauf["Datum"].max()
    fromdate =  maxdate - timedelta(ndays) if ndays is not None else None
    for selbl in blverlauf["Bundesland"].unique(): #("Österreich", "Wien", "Oberösterreich"): 
        fig, ax = plt.subplots()
        ax2, blvaext = draw_bl_ww_range(
            selbl, ax, logscale=logscale, fromdate=fromdate)
        cov.set_date_opts(ax, week_always=True)
        ax.set_ylabel("Genkopien pro Tag / EW * 10⁶")
        if ax2:
            ax2.set_ylabel("Fälle pro Woche / 100.000 EW")
        ax.set_title(ABW_TINFO + ", Daten bis " + blvaext.index[-1].strftime("%a %x"), y=1.09)
        #ax.set_xlim(*ax.get_xlim())
        fig.legend(ncol=3, frameon=False, loc="upper center", bbox_to_anchor=(0.5, 0.97))
        if logscale:
            fig.suptitle(selbl + ": SARS-CoV-2 im Abwasser, verschiedene Datenstände, logarithmische Skala", y=1.03)
        else:
            fig.suptitle(selbl + ": SARS-CoV-2 im Abwasser, verschiedene Datenstände", y=1.03)
        if ndays:
            ax.set_xlim(
                left=fromdate,
                right=maxdate + max(2, timedelta(ndays * 0.05)))
    #break

def draw_all_bl_ww(logscale=True, ndays=None, inz='scalefull'):
    sharey = not logscale
    fig, axs = plt.subplots(figsize=(10, 12), nrows=5, ncols=2, sharex=True, sharey=sharey)
    fig.subplots_adjust(wspace=0.3, hspace=0.17)
    maxdate = blverlauf["Datum"].max()
    fromdate = maxdate - timedelta(ndays)
    for selbl, ax in zip(list(cov.Bundesland.categories)[1:] + ["Österreich"], axs.flat): #("Österreich", "Wien", "Oberösterreich"): 
        ax2, blvaext = draw_bl_ww_range(
            selbl, ax, logscale=logscale, fromdate=fromdate if inz != 'scalefull' else None)
        ax.set_title(selbl + " bis " + blvaext.index[-1].strftime("%a %x"), y=0.96)
        #ax.set_xlim(*ax.get_xlim())
        if ax is axs.flat[0]:
            fig.legend(ncol=3, frameon=False, loc="upper center", bbox_to_anchor=(0.5, 0.97), title=ABW_TINFO)
        if ax in axs[-1]:
            if ndays:
                ax.set_xlim(
                    left=fromdate,
                    right=maxdate + timedelta(ndays * 0.05))
            cov.set_date_opts(ax, week_always=True, showday=False)
            if ax is axs.flat[-1] and not logscale and sharey:
                ax.set_ylim(bottom=0)


        ax.grid(axis="x", which="minor", lw=0.5)
        if ax in axs.T[0]:
            ax.set_ylabel("Kopien/Tag/EW*10⁶")
        if ax2 and ax in axs.T[-1]:
            ax2.set_ylabel("Fälle/Woche/EW*10⁵")
    seplabel = "" if sharey else "separate "
    scalabel = "Skala" if sharey else "Skalen"
    if logscale:
        fig.suptitle(selbl + f": SARS-CoV-2 im Abwasser, verschiedene Datenstände, {seplabel}logarithmische {scalabel}", y=0.99)
    else:
        fig.suptitle(selbl + f": SARS-CoV-2 im Abwasser, verschiedene Datenstände, {seplabel}{scalabel}", y=0.99)

    fig.autofmt_xdate()

    #draw_all_bl_ww()
draw_all_bl_ww(logscale=True, ndays=60, inz='scaledisplay')
#draw_each_bl_ww(logscale=True, ndays=(blvfull["Datum"].max() - pd.to_datetime("2023-02-15")).days + 1)

In [ ]:
#display(blv1)
def draw_blv1(blv1=blv1):
    blv1 = blv1[blv1.index.get_level_values("Datum") >= pd.to_datetime("2023-04-17")]
    ax = sns.lineplot(blv1["y"].reset_index(), x="Datum", y="y", hue="Bundesland", mew=0, marker=".")
    if True:
        cov.set_logscale(ax)
        ax.set_ylim(bottom=3, top=100)
    else:
        ax.set_ylim(bottom=0)
    ax.get_legend().remove()
    cov.set_date_opts(ax, blv1.index.get_level_values("Datum"), week_always=True)
    ax.set_ylabel("Genkopien pro Tag / EW * 10⁶")
    fig = ax.figure

    has_erstmeldung = blv1["FileDate"].max() == blvfull["FileDate"].max()
    fig.suptitle(
        "Österreich : SARS-CoV-2 im Abwasser, nur Erstmeldungen, logarithmische Skala",
        y=1.04 if has_erstmeldung else 1.06)
    fig.legend(
        *cov.sortedlabels(ax, blv1.reset_index(), "y", fmtval=lambda v: f"{v:.0f}"),
        ncol=5, frameon=False, loc="upper center", bbox_to_anchor=(0.5, 0.965))
    ax.set_title(
        ABW_TINFO +
        ("" if has_erstmeldung else
         "\nLetzte neue Erstmeldung " + blv1["FileDate"].max().strftime("%a %d.%m.%Y %H:%M")) +
        ", Daten bis " + blv1.index.get_level_values("Datum").max().strftime("%a %x"),
        y=1.09)
    cov.labelend2(ax, blv1.reset_index(), ycol="y", colorize=sns.color_palette(), ends=(True, True))
    #print(blv1["FileDate"].max())
draw_blv1()

# Krankenstand

* Daten von: Österreichischer Dachverband der Sozialversicherungsträger
* Daten-URL: <https://www.sozialversicherung.at/cdscontent/?contentid=10007.853001&portal=svportal>
* Daten-Aktualisierung: Monatlich, ca. ab 25.
* [Mehr Daten-Infos](https://github.com/zeitferne/covidat-data/blob/main/docs/sozialversicherung-monatsberichte.md)

In [ ]:
ks = pd.read_csv(util.COLLECTROOT / "sozialversicherung-monatsberichte/ks_all.csv",
                 sep=";", encoding="utf-8",
                 dtype={"date": pd.PeriodDtype("M")},
                 index_col=["date", "insurer", "employment"])

ks["jun_year"] = ks.index.get_level_values("date").asfreq("A-JUN").year - 1
ks["jun_year_l"] = ks["jun_year"].astype(str) + "/" + ((ks["jun_year"] + 1) % 100).astype(str) 
ks["jun_month"] = (ks.index.get_level_values("date").month - 1 + (12 - 6)) % 12 + 1
ks["cases_n"] = ks["cases"] / ks.index.get_level_values("date").days_in_month * 30
ks["m_start"] = ks.index.get_level_values("date").start_time

#ks["qyear"] = ks["qdate"].dt.qyear
ks = cov.add_sum_rows(ks, "employment", "Insgesamt", {
    "insured": "sum", "cases": "sum", "cases_n": "sum", "active_end": "sum",
    "jun_year": "first", "jun_year_l": "first", "jun_month": "first", "m_start": "first"})

ks["active_end_pp"] = ks["active_end"] / ks["insured"]
ks["cases_pp_n30"] = ks["cases_n"] / ks["insured"]
ks["cases_pp_raw"] = ks["cases"] / ks["insured"]
#ks_s["cases_pp_n30"] = ks_s["cases_n"] / ks_s["insured"]
#ks_s["active_end_pp"] = ks_s["active_end"] / ks_s["insured"]
#ks["m_start"] = ks0["date"].dt.start_time
#display(ks_s.query("insurer == 'Insgesamt' and employment != 'angest'")[["jun_year", "month"]])

In [ ]:
def plt_ks_yoy(y, bl, empl="Insgesamt"):
    whatlabel = (
        "Krankenstandfälle/Monat (30-Tage-normiert)" if y == "cases_pp_n30" else
        "Laufende Krankenstände am Monatsende" if y == "active_end_pp" else None)
    empllabel = "" if empl == "Insgesamt" else f" (nur {empl})"
    fig, ax = plt.subplots()
    #display(ks.dtypes)
    #ax.set_xlabels(["])
    blq = "Insgesamt" if bl == 'Österreich' else bl
    pdata = ks.query(f"insurer == '{blq}' and employment == '{empl}'").sort_index().reset_index()
    pal = sns.color_palette("tab10", n_colors=pdata["jun_year_l"].nunique())[::-1]
    pltargs = dict(x="jun_month", y=y, hue="jun_year_l", palette=pal, sort=False, markersize=9)
    sns.lineplot(pdata,
                  #lw=3,
                #size=pdata["jun_year_l"].map(lambda y: 3 if y == pdata.iloc[0]["jun_year_l"] else 2),
                #sizes=(2, 4),
                marker="o" if y != "active_end_pp" else None,
                lw=3, **pltargs)
    if y == "active_end_pp":
        wday = pdata["date"].dt.end_time.dt.dayofweek
        is_weekend = wday > 4
        is_monday = wday == 0
        pd0 = pdata.copy()
        pd0.loc[is_monday | is_weekend, y] = np.nan
        sns.lineplot(
            pd0,
            marker="o",
            lw=0, **pltargs, legend=False)
        pd0 = pdata.copy()
        pd0.loc[~is_monday, y] = np.nan
        sns.lineplot(
            pd0,
            marker="X",
            lw=0, **pltargs, legend=False, mew=0)
    
    cov.set_percent_opts(ax)
    ax.axvline(12 - 6 + 1 - 0.5, color="k", ls="--", lw=1, zorder=1)
    ax.set_xticks(np.arange(1, 12+1))
    ax.set_xticklabels(["Jul", "Aug", "Sep", "Okt", "Nov", "Dez", "Jän", "Feb", "Mär", "Apr", "Mai", "Jun"])
    ax.set_ylim(bottom=0)
    ax.set_xlabel("Monat")
    ax.legend(title="Jahr bzw. Saison")
    #ax.set_ylabel("Anteil der Versicherten (30-Tage-normiert)")
    ax.set_ylabel("Fälle/Versicherte")
    #fig.suptitle("Aktive Krankenstandfälle am Monatsende als Anteil der Versicherten", y=0.93)
    fig.suptitle(f"{bl}{empllabel}: {whatlabel} als Anteil der Versicherten", y=0.93)
    kwargs = dict(ax=ax, mms=pdata, ycol=y, cats="jun_year", x="jun_month", colorize=pal)
    cov.labelend2(**kwargs, shorten=lambda x: str((x + 1) % 100))
    cov.labelend2(**kwargs, ends=(True, False), shorten=lambda x: "17" if x == 2016 else str(x % 100))
    ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(0.02))
    cov.stampit(fig, "Monatsberichte des österr. Dachverband der Sozialversicherungsträger")
    ax.legend(ncol=3, title="Jahr bzw. Saison")
plt_ks_yoy("active_end_pp", "Österreich")
if False:
    for bl in cov.BUNDESLAND_BY_ID.values():
        plt_ks_yoy("cases_pp_n30", bl)
plt_ks_yoy("cases_pp_n30", "Österreich", "Insgesamt")
plt_ks_yoy("cases_pp_n30", "Österreich", "ArbeiterInnen")
plt_ks_yoy("cases_pp_n30", "Österreich", "Angestellte")

In [ ]:
def plt_ks_bl_all():
    fig, ax = plt.subplots()
    ks0 = ks.query("employment == 'Insgesamt'").reset_index()#.query("employment == 'angest'").reset_index()
    sns.lineplot(ks0.query(
        "insurer in ('Vorarlberg', 'Tirol', 'Salzburg', 'Oberösterreich', 'Kärnten', 'Steiermark', 'Niederösterreich', 'Wien', 'Burgenland')"
    ), hue="insurer",# style="employment",
                 x="m_start", y="cases_pp_n30", marker=".", mew=0)
    cov.set_percent_opts(ax, decimals=1)
    ax.xaxis.set_major_locator(matplotlib.dates.YearLocator())
    ax.xaxis.set_minor_formatter(matplotlib.dates.DateFormatter("%b"))
    ax.xaxis.set_minor_locator(matplotlib.dates.MonthLocator([1, 4, 7, 10]))
    ax.set_ylim(bottom=0)
    
    ax.grid(which="minor", lw=0.3)
    #fig.autofmt_xdate()
    ax.tick_params(axis="x", which="major", pad=11)
    #cov.set_logscale(ax)
    ax.legend(ncol=2, loc="lower left")
    fig.suptitle("Krankenstandzugang/Monat (normiert auf 30 Tage) in % der Versicherten")
    ax.set_title("Arbeiter + Angestellte, je Versichernder ÖGK")
    ax.set_xlabel("Berichtsmonat")
    ax.set_ylabel("Fälle/Versicherte (30-Tage-normiert)")
plt_ks_bl_all()

In [ ]:
def plt_emp_ks_all():
    logscale = False
    fig, ax = plt.subplots()
    if logscale:
        cov.set_logscale(ax)
    fig.suptitle("Krankenstandzugang/Monat (normiert auf 30 Tage) in % der Versicherten")
    ax.set_title("Über alle im Bericht erfassten Träger, je Arbeitsverhältnis")
    
    pdata = ks.query("insurer == 'Insgesamt'").reset_index()
    pargs = dict(x="m_start", y="cases_pp_n30", estimator=None, marker=".", mew=0, ax=ax)
    ax = sns.lineplot(pdata.query("employment != 'Insgesamt'"), hue="employment", ls="--", **pargs)
    sns.lineplot(pdata.query("employment == 'Insgesamt'"),
                 lw=2, markersize=10, color="grey", label="Insgesamt", **pargs)
    cov.set_percent_opts(ax, decimals=1)
    ax.xaxis.set_major_locator(matplotlib.dates.YearLocator())
    ax.xaxis.set_minor_formatter(matplotlib.dates.DateFormatter("%b"))
    ax.xaxis.set_minor_locator(matplotlib.dates.MonthLocator([1, 4, 7, 10]))
    ax.grid(which="minor", lw=0.3)
    #fig.autofmt_xdate()
    ax.tick_params(axis="x", which="major", pad=11)
    if not logscale:
        ax.set_ylim(bottom=0)
    
    ax.set_xlabel("Berichtsmonat")
    ax.set_ylabel("Fälle/Versicherte (30-Tage-normiert)")
    
    ax.legend()
plt_emp_ks_all()

# Influenza und Atemwegserkrankungen

* Daten von: Agentur für Gesundheit und Ernährungssicherheit (AGES), Zentrum für Virologie der Medizinischen Universität Wien,  Diagnostisches Influenzanetzwerk Österreich (DINÖ)
* Daten-URLs:
  * <https://www.ages.at/mensch/krankheit/krankheitserreger-von-a-bis-z/grippe>
  * <https://www.virologie.meduniwien.ac.at/wissenschaft-forschung/virus-epidemiologie/influenza-projekt-diagnostisches-influenzanetzwerk-oesterreich-dinoe/>
* Daten-Aktualisierung: Wöchentlich; Daten werden nur ca. ab Kalenderwoche 40 (Anfang Oktober) bis Ende der Grippewelle oder max. ca. KW 20 (Mitte Mai) aktualisiert

In [ ]:
def onlyelem(xs):
    xs = iter(xs)
    v = next(xs)
    try:
        next(xs)
    except StopIteration:
        return v
    raise ValueError("More than one element")
grippe = onlyelem(pd.read_html(
    util.DATAROOT / "ages-epi-misc/grippe_20230714_090503.html",
    match="Influenza A/H3N2 Fall",
    thousands=".",
    decimal=","))

In [ ]:
def plt_flu_posrate():
    gfallcol = [c for c in grippe.columns if " Fall" in c]
    grippe["Woche"] = grippe["Woche"].astype(str)
    grippe.set_index("Woche", inplace=True, verify_integrity=True)
    npos = grippe[gfallcol].sum(axis=1)
    ntest = (npos / grippe["Positiv-Rate (positive Proben/Untersuchte Proben)"]).round()
    ppos = grippe[gfallcol].div(ntest, axis=0)
    #display(ntest)
    #display(grippe[gfallcol])
    ppos.rename(columns={c: '% ' + c.replace(" Fall", "-positiv") for c in ppos.columns}, inplace=True)
    color = [f"C{i}" for i in range(len(ppos.columns))]
    ax = ppos.plot(color=color)
    ppos.where(lambda x: x != 0).plot(ax=ax, lw=0, marker="o", markersize=10, alpha=0.8, color=color, legend=False)
    fig = ax.figure
    ax.set_ylim(bottom=0)
    cov.set_percent_opts(ax)
    ax.set_ylabel("Positivrate")
    fig.suptitle("Influenza in Österreich, Sentinel-Proben 2022/23 (AGES/DINÖ)", y=0.93)
plt_flu_posrate()

In [ ]:
def plt_flu_posrate_and_cnt():
    ax = grippe.plot(y=gfallcol, marker="o", markersize=10, alpha=0.8)
    cov.set_logscale(ax)
    ax2 = ax.twinx()
    grippe.plot(y=[c for c in grippe.columns if "Positiv-Rate" in c], ax=ax2, color="k", marker="s",
               markersize=4)
    ax2.grid(False)
    ax2.get_legend().remove()
    ax.get_legend().remove()
    fig = ax.figure
    fig.suptitle("Influenza in Österreich, Sentinel-Proben 2022/23 (AGES/DINÖ)", y=1.05)
    fig.legend(loc="upper center", ncol=2, frameon=False, bbox_to_anchor=(0.5, 1))
    ax.set_ylabel("Anzahl positiver Proben (log)")
    ax2.set_ylabel("Positivrate")
    cov.set_percent_opts(ax2)
    ax.set_ylim(bottom=0.9)
    ax2.set_ylim(bottom=0)
plt_flu_posrate_and_cnt()

In [ ]:
def plt_ntest():
    (ntest).plot(marker="o").set_ylim(0)
    plt.gcf().suptitle("Anzahl Sentinel-Proben (positiv+negativ) / KW (geschätzt)", y=0.93)
plt_ntest()